In [1]:
import pipeline_config

In [2]:
pipeline_config

<module 'pipeline_config' from '/home/jupyter/src/TrainingPipeline/pipeline_config.py'>

In [18]:
PROJECT_ID = "ritheesh-1733201347"
BUCKET_NAME = "stallion-data"
PIPELINE_ROOT = f"gs://vertex-pipeline-roots"
DATA_PATH = "merged_dataset.csv"
LOCATION = "us-central1"

In [19]:
from kfp import dsl
from kfp.compiler import Compiler
from google.cloud import aiplatform
import logging
from datetime import datetime
from google_cloud_pipeline_components.v1.custom_job import create_custom_training_job_from_component
from components.preprocess_component import preprocess_component as PreprocessOp
from components.dataloader_component import dataloader_component as DataloaderOp

import pipeline_config

In [20]:
preprocessOP=create_custom_training_job_from_component(PreprocessOp,
    display_name='PreProcessing Pipeline',
    machine_type=pipeline_config.ComputeResources.PREPROCESS_MACHINE_TYPE,
)

In [21]:
@dsl.pipeline(
    name="tft-training-pipeline",
    description="End to end pipeline for training the Temporal Fusion Transformer model"
)
def training_pipeline(
    project: str,
    location: str,
    gcs_bucket: str,
    gcs_path: str
):
    """Main pipeline for training and inference"""
    
    special_days = [
        "easter_day",
        "good_friday",
        "new_year",
        "christmas",
        "labor_day",
        "independence_day",
        "revolution_day_memorial",
        "regional_games",
        "fifa_u_17_world_cup",
        "football_gold_cup",
        "beer_capital",
        "music_fest"
    ]
    
    #pipeline components
    preprocess_task = preprocessOP(
        project=project,
        location=location,
        gcs_bucket=gcs_bucket,
        gcs_path=gcs_path,
        special_days=special_days
    )

In [22]:
#compiling the pipeline
compiler = Compiler()
compiler.compile(
    pipeline_func=training_pipeline,
    package_path="tft_training_pipeline_latestt.json"
)

In [23]:
#creating and running the training pipeline
training_job = aiplatform.PipelineJob(
    display_name="tft-training-pipeline",
    template_path="tft_training_pipeline_latestt.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        "project": PROJECT_ID,
        "location": LOCATION,
        "gcs_bucket": BUCKET_NAME,
        "gcs_path": DATA_PATH
    }
)

In [24]:
aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [25]:
training_job.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/329178879731/locations/us-central1/pipelineJobs/tft-training-pipeline-20250128153643
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/329178879731/locations/us-central1/pipelineJobs/tft-training-pipeline-20250128153643')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/tft-training-pipeline-20250128153643?project=329178879731


RuntimeError: Job failed with:
code: 9
message: " The DAG failed because some tasks failed. The failed tasks are: [preprocess-component].; Job (project_id = ritheesh-1733201347, job_id = 9151727584310984704) is failed due to the above error.; Failed to handle the job: {project_number = 329178879731, job_id = 9151727584310984704}"
